## 1. Configuração e Carregamento

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Configurações ---
K = 3 # Recomendar o Top 3

print("Bibliotecas carregadas.")

Bibliotecas carregadas.


## 2. Carregar os Dados

In [3]:
# Carregar os nossos ficheiros de "combustível"
try:
    df_provincias_feat = pd.read_csv('../data/provincia_features_MVP.csv', index_col='Provincia')
    df_users_feat = pd.read_csv('../data/user_features_MVP.csv', index_col='user_id')
    df_interactions = pd.read_csv('../data/user_interactions_MVP.csv')
    print("Ficheiros MVP carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Ficheiros MVP não encontrados na pasta /data/!")

print("\n--- Features das Províncias (Amostra) ---")
# Vamos usar APENAS as features de "match", ignorando popularidade e custo
features_das_provincias = df_provincias_feat[['impacto_ambiental', 'feat_cultura', 'feat_praia', 'feat_aventura', 'feat_natureza']]
print(features_das_provincias.head(3))

print("\n--- Features dos Utilizadores (Amostra) ---")
# Vamos alinhar as features: 'pref_sustentavel' deve fazer 'match' com 'impacto_ambiental'
features_dos_utilizadores = df_users_feat[['pref_sustentavel', 'pref_cultura', 'pref_praia', 'pref_aventura', 'pref_natureza']]
print(features_dos_utilizadores.head(3))

Ficheiros MVP carregados com sucesso.

--- Features das Províncias (Amostra) ---
           impacto_ambiental  feat_cultura  feat_praia  feat_aventura  \
Provincia                                                               
Luanda                   0.2           1.0         0.8            0.1   
Benguela                 0.4           0.5         1.0            0.5   
Huíla                    0.7           0.7         0.0            0.8   

           feat_natureza  
Provincia                 
Luanda               0.2  
Benguela             0.3  
Huíla                1.0  

--- Features dos Utilizadores (Amostra) ---
         pref_sustentavel  pref_cultura  pref_praia  pref_aventura  \
user_id                                                              
1                     0.1           1.0         0.8            0.1   
2                     0.9           0.0         0.0            1.0   
3                     0.8           0.9         0.0            0.5   

         pref_natureza

## 3. O "Cérebro" do Modelo (Similaridade de Cosseno)

In [4]:
# Garantir que as colunas estão na mesma ordem para a matemática funcionar
# (Renomeamos a 'pref_sustentavel' do utilizador para 'impacto_ambiental' para o 'match')
features_dos_utilizadores.columns = features_das_provincias.columns

# 1. Calcular a Matriz de Similaridade
# O "coração" do Content-Based: (Utilizadores x Features) X (Features x Províncias)
matriz_similaridade = cosine_similarity(
    features_dos_utilizadores.values, 
    features_das_provincias.values
)

# 2. Converter num DataFrame legível
df_scores_cb = pd.DataFrame(
    matriz_similaridade,
    index=features_dos_utilizadores.index,
    columns=features_das_provincias.index
)

print("\n--- MATRIZ DE SCORES (Similaridade de Cosseno) ---")
print(df_scores_cb.round(2))


--- MATRIZ DE SCORES (Similaridade de Cosseno) ---
Provincia  Luanda  Benguela  Huíla  Namibe  Malanje  Cabinda  Huambo  \
user_id                                                                
1            1.00      0.84   0.50    0.56     0.54     0.71    0.60   
2            0.22      0.53   0.89    0.84     0.86     0.73    0.80   
3            0.64      0.63   0.96    0.72     0.97     0.86    0.99   

Provincia  Cuando Cubango  
user_id                    
1                    0.31  
2                    0.98  
3                    0.87  


## 4. Gerar Recomendações Personalizadas

In [8]:
def get_cb_recommendations(user_id, scores_matrix, k):
    """
    Retorna as K províncias com maior score para um dado utilizador.
    """
    # Obter os scores desse utilizador
    user_scores = scores_matrix.loc[user_id]
    
    # Ordenar do maior para o menor e obter o Top-K
    recommendations = user_scores.sort_values(ascending=False).index.tolist()
    return recommendations[:k]

# Gerar recomendações para cada utilizador
recomendacoes_cb = {}
for user_id in df_users_feat.index:
    recomendacoes_cb[user_id] = get_cb_recommendations(user_id, df_scores_cb, K)

print("\n--- Recomendações Personalizadas (Content-Based) ---")
for user, recs in recomendacoes_cb.items():
    print(f"User {user}: {recs}")


--- Recomendações Personalizadas (Content-Based) ---
User 1: ['Luanda', 'Benguela', 'Cabinda']
User 2: ['Cuando Cubango', 'Huíla', 'Malanje']
User 3: ['Huambo', 'Malanje', 'Huíla']


## 5. Avaliação (Onde provamos o domínio)

In [7]:
# 1. Obter o "Ground Truth" (O que os utilizadores REALMENTE gostaram - ratings >= 4)
ground_truth = {}
for user in df_interactions['user_id'].unique():
    user_likes = df_interactions[
        (df_interactions['user_id'] == user) & (df_interactions['rating'] >= 4)
    ]['provincia_visitada'].tolist()
    ground_truth[user] = user_likes

# 2. Calcular Métricas de Relevância (Precisão, Recall)
lista_precision = []
lista_recall = []

for user, likes in ground_truth.items():
    if not likes or user not in recomendacoes_cb:
        continue # Ignorar utilizadores sem 'likes' ou sem recs

    recs_do_user = recomendacoes_cb[user]
    hits = len(set(recs_do_user) & set(likes))
    
    precision_k = hits / K
    lista_precision.append(precision_k)
    
    recall_k = hits / len(likes)
    lista_recall.append(recall_k)

print("\n--- Métricas de Relevância (Média) ---")
print(f"Precisão@ {K} (Média): {np.mean(lista_precision) * 100:.2f}%")
print(f"Recall@ {K} (Média):   {np.mean(lista_recall) * 100:.2f}%")

# 3. Calcular Métricas de Plataforma e Impacto
mainstream_list = df_provincias_feat.sort_values(by='popularidade', ascending=False).index[:2].tolist()
sustainable_list = df_provincias_feat[df_provincias_feat['impacto_ambiental'] >= 0.7].index.tolist()

lista_diversidade = []
lista_sustentabilidade = []

for user, recs in recomendacoes_cb.items():
    emergentes_recomendados = len(set(recs) - set(mainstream_list))
    sustentaveis_recomendados = len(set(recs) & set(sustainable_list))
    
    lista_diversidade.append(emergentes_recomendados / K)
    lista_sustentabilidade.append(sustentaveis_recomendados / K)

print("\n--- MÉTRICAS DE PLATAFORMA E IMPACTO (MÉDIA) ---")
print(f"Diversidade@ {K} (% Emergentes):   {np.mean(lista_diversidade) * 100:.2f}%")
print(f"Sustentabilidade@ {K} (% Sustentáveis): {np.mean(lista_sustentabilidade) * 100:.2f}%")


--- Métricas de Relevância (Média) ---
Precisão@ 3 (Média): 66.67%
Recall@ 3 (Média):   100.00%

--- MÉTRICAS DE PLATAFORMA E IMPACTO (MÉDIA) ---
Diversidade@ 3 (% Emergentes):   77.78%
Sustentabilidade@ 3 (% Sustentáveis): 66.67%


## 6. Conclusões: A Solução de "Match"

O Modelo Baseado em Conteúdo (CB) foi projectado para vencer o Baseline, focando-se em "match" de features (`pref_sustentavel` -> `impacto_ambiental`) em vez de popularidade. Os resultados são uma vitória clara e defensável.

### 1. Comparação de Métricas: Baseline vs. Content-Based

| Métrica | 02_Baseline (Popularidade) | 03_Content-Based (NongoTour) | Mudança (Delta) |
| :--- | :---: | :---: | :---: |
| **`Precisão@3`** | 22.22% | **66.67%** | **+44.45 pts** |
| **`Recall@3`** | 33.33% | **100.00%** | **+66.67 pts** |
| **`Diversidade@3`** | 33.33% | **77.78%** | **+44.45 pts** |
| **`Sustentabilidade@3`** | 33.33% | **66.67%** | **+33.34 pts** |

### 2. Análise dos Resultados

1.  **Sucesso na Relevância (Precisão/Recall):** O `Recall@3` atingiu **100%**. Isto significa que *todos os itens* que os utilizadores gostaram (no nosso set de teste) estavam na lista do Top 3. A `Precisão@3` (66.67%) prova que 2 em cada 3 recomendações foram "acertos" perfeitos. O modelo recomendou `['Malanje', 'Cuando Cubango']` ao User 2 e `['Malanje', 'Huambo']` ao User 3, provando que a personalização funciona.

2.  **Sucesso na Plataforma e Impacto (Diversidade/Sustentabilidade):** Vencemos o Baseline em mais de 44 pontos na `Diversidade@3` e duplicámos a `Sustentabilidade@3`. O modelo recomendou activamente destinos emergentes e sustentáveis (`Cuando Cubango`, `Malanje`, `Huambo`, `Cabinda`) porque eles eram um "match" para os perfis dos utilizadores.

**Fraqueza/Ponto Cego (A Ser Resolvido):**
Este modelo é excelente, mas é "limitado". Ele *nunca* recomendará algo fora do perfil explícito do utilizador (sofre de "serendipidade zero"). Ele não *aprende* com o comportamento, apenas com os perfis que fornecemos.

**Próximo Passo:**
Vamos construir o **`04_Collaborative_Filtering.ipynb`**. Este modelo de "ML Puro" vai ignorar os perfis e *aprender* padrões de comportamento directamente dos *ratings*, usando SVD.